In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment
import warnings


def scrape_player_stats(player_name, url, table_id="stats_standard", season=None):
    warnings.filterwarnings("ignore")
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find("table", id =table_id) if table_id else soup.find("table")
    if not table:
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            if 'stats_standard' in comment:
                comment_soup = BeautifulSoup(comment, 'html.parser')
                table = comment_soup.find('table', id='stats_standard')
                thead = table.find('thead')
                theader = thead.find_all('th')
                columns = [th.text.strip() for th in theader.find_all('th')]
                n = 7
                sub_columns = columns[n:]
                tbody = table.find('tbody')
                rows = tbody.find_all('tr')
                f = []
                for row in rows:
                    cells = row.find_all(['td', 'th'])
                    if len(cells) > 0:
                        data = [cell.text.strip() for cell in cells]
                        f.append(data)
                df2 = pd.DataFrame(f, columns=sub_columns)
            

                if not table:
                    print(f"Table not found for {player_name}")
                    return None

                df = pd.read_html(str(table), header=1)[0]
                df['League'] = player_name
                #df['Season'] = season

                return df2


'''
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        if 'stats_standard' in comment:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            table = comment_soup.find('table', id='stats_standard')
'''

"\n    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):\n        if 'stats_standard' in comment:\n            comment_soup = BeautifulSoup(comment, 'html.parser')\n            table = comment_soup.find('table', id='stats_standard')\n"

In [6]:
#https://fbref.com/en/comps/20/stats/Bundesliga-Stats
years = ["2024-2025","2023-2024","2022-2023","2021-2022","2020-2021","2019-2020","2018-2019","2017-2018","2016-2017","2015-2016"]
leagueinfo = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/stats/{year}-Bundesliga-Stats"
    },
    {
        "name": "Premier League",
        "url": "https://fbref.com/en/comps/9/{year}/stats/{year}-Premier-League-Stats"
    },
    {
        "name": "La Liga",
        "url": "https://fbref.com/en/comps/12/{year}/stats/{year}-La-Liga-Stats"
    },
    {
        "name": "Serie A",
        "url": "https://fbref.com/en/comps/11/{year}/stats/{year}-Serie-A-Stats"
    },
    {
        "name": "Ligue 1",
        "url": "https://fbref.com/en/comps/13/{year}/stats/{year}-Ligue-1-Stats"
    },
    {
        "name": "Eredivisie",
        "url": "https://fbref.com/en/comps/23/{year}/stats/{year}-Eredivisie-Stats"
    },
    {
        "name": "Primeira Liga",
        "url": "https://fbref.com/en/comps/32/{year}/stats/{year}-Primeira-Liga-Stats"
    },
    {
        "name": "Championship",
        "url": "https://fbref.com/en/comps/10/{year}/stats/{year}-EFL-Championship-Stats"
    },
    {
        "name": "Scottish Premiership",
        "url": "https://fbref.com/en/comps/40/{year}/stats/{year}-Scottish-Premiership-Stats"
    }
 
]

leagues = []
for year in years:
    for league in leagueinfo:
        leagues.append({
            "name": f"{league['name']} {year}",
            "url": league["url"].format(year=year),
            "season": year.replace("-", "/"),
        })

league_dfs = {}
combined_list = []
for league in leagues:
    df = scrape_player_stats(league["name"], league["url"])
    if df is not None:
        league_dfs[league["name"]] = df
        combined_list.append(df)

AttributeError: ResultSet object has no attribute "find_all". You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
df = pd.concat(combined_list,ignore_index=True)
df['League'].value_counts()

League
Serie A 2024-2025          659
Serie A 2019-2020          624
Serie A 2016-2017          605
La Liga 2019-2020          592
Primeira Liga 2019-2020    584
Ligue 1 2024-2025          575
Eredivisie 2015-2016       491
Name: count, dtype: int64

In [ ]:
league_dfs =  pd.concat(combined_list, ignore_index=True)
league_dfs['League'].unique().tolist()

['Serie A 2024-2025',
 'Ligue 1 2024-2025',
 'La Liga 2019-2020',
 'Serie A 2019-2020',
 'Primeira Liga 2019-2020',
 'Serie A 2016-2017',
 'Eredivisie 2015-2016']

In [ ]:
df = pd.concat(combined_list, ignore_index=True)
df['League'].unique().tolist()


['Serie A 2024-2025',
 'Ligue 1 2024-2025',
 'La Liga 2019-2020',
 'Serie A 2019-2020',
 'Primeira Liga 2019-2020',
 'Serie A 2016-2017',
 'Eredivisie 2015-2016']